In [ ]:
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

# Input files' directory
# print("Input files: ")
# list.files(path = "../input")

library(data.table)
library(tidyverse)
library(tidyr)
library(lubridate)
library(corrplot)
library(mice)
library(DMwR)

In [ ]:
#----------------- ZILLOW'S HOME VALUE ESTIMATE (via LINEAR/MULTIPLE REGRESSION METHOD) ---------------

# We are going to perform EDA on the Zillow's Home Value Estimate. 
# The following are good steps to begin the process:
# 1. Import the data into a variable. (Load train_201 from .../input folder) 
# 2. Check the dimensions of the 
zillow_prop_2017 <- fread("../input/properties_2017.csv")
paste("Dimensions of complete_df: " , toString(dim(zillow_prop_2017)))

zillow_dataframe_2017 <- fread("../input/train_2017.csv")
paste("Dimensions of training_samples: " , toString(dim(zillow_dataframe_2017)))

# We have decided to go ahead with 2017 data because it being recent. We will further investigate `train_2017.csv` and `properties_2017`
# to get more insight to the data.

In [ ]:
# We will look at the couple of head data to know the data better. 
# Obs: We can observe that the `train_2017.csv` file contains the training data 
#      with mentioned logerror and transaction data, while the `properties_2017.csv`
#      contains the records and its attributes for training the model.

#      We will need to club/merge both the files (natural_join) to get our training data. 
#      (We will do that after cleaning the data)

head(zillow_dataframe_2017, n=15)
head(zillow_prop_2017, n=15)

In [ ]:
# To clean the data, let us first rename the column names to our convenience.
# 
zillow_prop_2017 <- zillow_prop_2017 %>% rename(
  id_parcel =  parcelid,
  build_year =  yearbuilt,
  area_basement =  basementsqft,
  area_patio =  yardbuildingsqft17,
  area_shed =  yardbuildingsqft26, 
  area_pool =  poolsizesum,  
  area_lot =  lotsizesquarefeet, 
  area_garage =  garagetotalsqft,
  area_firstfloor_finished =  finishedfloor1squarefeet,
  area_total_calc =  calculatedfinishedsquarefeet,
  area_base =  finishedsquarefeet6,
  area_live_finished =  finishedsquarefeet12,
  area_liveperi_finished =  finishedsquarefeet13,
  area_total_finished =  finishedsquarefeet15,  
  area_unknown =  finishedsquarefeet50,
  num_unit =  unitcnt, 
  num_story =  numberofstories,  
  num_room =  roomcnt,
  num_bathroom =  bathroomcnt,
  num_bedroom =  bedroomcnt,
  num_bathroom_calc =  calculatedbathnbr,
  num_bath =  fullbathcnt,  
  num_75_bath =  threequarterbathnbr, 
  num_fireplace =  fireplacecnt,
  num_pool =  poolcnt,  
  num_garage =  garagecarcnt,  
  region_county =  regionidcounty,
  region_city =  regionidcity,
  region_zip =  regionidzip,
  region_neighbor =  regionidneighborhood,  
  tax_total =  taxvaluedollarcnt,
  tax_building =  structuretaxvaluedollarcnt,
  tax_land =  landtaxvaluedollarcnt,
  tax_property =  taxamount,
  tax_year =  assessmentyear,
  tax_delinquency =  taxdelinquencyflag,
  tax_delinquency_year =  taxdelinquencyyear,
  zoning_property =  propertyzoningdesc,
  zoning_landuse =  propertylandusetypeid,
  zoning_landuse_county =  propertycountylandusecode,
  flag_fireplace =  fireplaceflag, 
  flag_tub =  hashottuborspa,
  quality =  buildingqualitytypeid,
  framing =  buildingclasstypeid,
  material =  typeconstructiontypeid,
  deck =  decktypeid,
  story =  storytypeid,
  heating =  heatingorsystemtypeid,
  aircon =  airconditioningtypeid,
  architectural_style = architecturalstyletypeid
)

columnName_df <- colnames(zillow_prop_2017)
columnName_df

zillow_dataframe_2017 <- zillow_dataframe_2017 %>% rename(
  id_parcel = parcelid,
  date = transactiondate
)

columnName_train <- colnames(zillow_dataframe_2017)
columnName_train

In [ ]:
# We will join the data from both the files to get a training dataset.
complete_training_data <- zillow_dataframe_2017 %>% inner_join(zillow_prop_2017, by="id_parcel")
complete_training_data

In [ ]:
# On renaming the columns we are now a little comfortable with the data. 
# We will now find the percent of missing entries in all the columns 
# and then use the columns with less than 40% of missing data.
# Reason: The graph clearly shows that we do not need the data
# less than 40%. It reduces our number of columns significantly.

#Finding null values 
null_data_percent <- map_dbl(complete_training_data, function(x) { round((sum(is.na(x)) / length(x)) * 100, 2) })
data.frame(percent = null_data_percent, var = names(null_data_percent), row.names = NULL) %>%
    ggplot(aes(x = reorder(var, -percent), y = percent)) +
    geom_bar(stat = 'identity', fill = 'blue') +
    labs(x = '', y = 'Missing Data %', title = '% missing data by all feature') +
    theme(axis.text.x = element_text(angle = 90, hjust = 1))

#Finding <40% null values
null_data_percent_more_than_40 <- null_data_percent[null_data_percent > 40]
null_data_percent_less_than_40 <- null_data_percent[null_data_percent <= 40]

data.frame(percent = null_data_percent_less_than_40, var = names(null_data_percent_less_than_40), row.names = NULL) %>%
    ggplot(aes(x = reorder(var, -percent), y = percent)) +
    geom_bar(stat = 'identity', fill = '#4169E1') +
    labs(x = '', y = 'Missing Data %', title = '% missing data by feature (less than 40%)') +
    theme(axis.text.x = element_text(angle = 90, hjust = 1))

names(null_data_percent_less_than_40)

In [ ]:
#removing the columns with more than 40% missing data. 
complete_training_data[,c(names(null_data_percent_more_than_40))] <- NULL
paste("Dimensions of df: " , toString(dim(zillow_prop_2017)))

# complete_training_data_clean <- na.exclude(complete_training_data)
# dim(complete_training_data_clean)

In [ ]:
#To save the checkpoint for the dataframe and aviod running the program again and again.
total_training <- complete_training_data
total_training

In [ ]:
date_col <- as_datetime((total_training$date))
tmp <- total_training %>% mutate(year_month = make_date(year = year(date_col), month = month(date_col)))
tmp %>% 
  group_by(year_month) %>% count() %>% 
  ggplot(aes(x = year_month,y = n)) +
  geom_bar( stat="identity", fill="#6A5ACD") +
  geom_vline(aes(xintercept = as.numeric(as.Date("2016-10-01"))),size=2) +
  labs(x = 'Month-Year', y = 'Count', title = 'Count of Records v/s Time Graph')

# This shows that there is more empty data than the needed data, and does not have much impact on the final result.
total_training %>%
  group_by(total_training$tax_delinquency) %>% count()
total_training$tax_delinquency <- NULL

In [ ]:
# Changing the string `date` column to type `Date`
mapStringToDate <- function(stringDate) {
  parse_date_time(stringDate, orders = c("ymd"))
}
total_training$date <- sapply(total_training$date, mapStringToDate)

In [ ]:
# We looked up a lot of kernels for managing the null values. 
# This is the best that we could come up for handling null values.

# Replacing the null values with zero, this is not the best technique. 
# But the data is categorised anyway, and it could have been replaced using KNN techniques
# but we have categoried it as 0 instead.
categories_zoning_landing_county <- unique(total_training$zoning_landuse_county)
zoning_landing_county_col <- total_training$zoning_landuse_county
for (x in 1:length(zoning_landing_county_col)) {
  temp <- match(zoning_landing_county_col[x], categories_zoning_landing_county)
  if (is.na(temp)) {
    temp <- 0
  }
  zoning_landing_county_col[x] = temp
}
total_training$zoning_landuse_county <- as.numeric(zoning_landing_county_col)

#same goes heree
categories_zoning_property <- unique(total_training$zoning_property)
zoning_property_col <- total_training$zoning_property
for (x in 1:length(zoning_property_col)) {
  temp <- match(zoning_property_col[x], categories_zoning_property)
  if (is.na(temp)) {
    temp <- 0
  }
  zoning_property_col[x] = temp
}
total_training$zoning_property <- as.numeric(zoning_property_col)

In [ ]:
# Every id_parcel is linked to location information. There are plenty of geolocation data attributes that can be used.
# These features can be condensed to form clusters - 'latitude', 'longitude', 'fips', 'regionidcounty', 'regionidcity', 'regionidzip', 'regionidneighborhood'

geographic_attributes <- c("latitude", "longitude", "fips", "region_county", "region_city", "region_zip")

# summary
total_training %>% 
  select_(.dots = geographic_attributes) %>% 
  summary()

# On further investigation, we can fill the NA's (as they are same number in latitude, longitude, fips,region_county). 
# First, we will re-create the original latitude & longitude by multipling it by 10e6.

total_training$latitude = total_training$latitude/1000000
total_training$longitude = total_training$longitude/1000000

# In addition to that, we know that latitude and longitude are a unique combination. We will keep the location point
# that has minimum null entries in the location attributes.

# Create unique location point of lat_long
total_training$pos = paste0(total_training$longitude,"_", total_training$latitude)

# Subset the location attributes from total_training dataframe
geolocation.dt <- setDT(total_training[, c(geographic_attributes, "pos")])

# Keep rows that contain most complete lat&long information
# Calculate for each row the number of non-missing values (count) and then 
# just keep the rows where "count"" is equal to the max for that group.
coordinates = copy(geolocation.dt)
coordinates = coordinates[, count := rowSums(!is.na(geolocation.dt))]
coordinates = coordinates[ , max.count := max(count, na.rm = TRUE), 
                           by = "pos"][count == max.count,.(region_county, region_city, region_zip, latitude, longitude, pos)]

coordinates = as.data.frame(coordinates)
coordinates = coordinates [!duplicated(coordinates$pos),]

# Next, we will remove location attributes from the total_training dataframe and join with the coordinates data:
# Remove location attributes, except latitude and longitude
total_training = setDT(total_training)
total_training[ , ':=' (region_county = NULL, 
                         region_city = NULL, 
                         region_zip = NULL,
                         pos = NULL),]
coordinates = setDT(coordinates)

# Merge with the coordinates data
total_training = coordinates[total_training, ,on = c("latitude", "longitude")]
total_training = as.data.frame(total_training)

In [ ]:
# Now that we have a more reliable geo-location information, we will next replace the tails of the lat & long distribution with means for the whole dataset: 

#  Find longitude distribution tails 
longitude    <- total_training$longitude
outliers_lon <- quantile(longitude, c(0.001,0.999), na.rm = TRUE)
xlon         <- which(!(longitude > outliers_lon[1] & longitude < outliers_lon[2]))

# Find latitude distribution tails 
latitude     <- total_training$latitude
outliers_lat <- quantile(latitude, c(0.001,0.999), na.rm = TRUE)
xlat         <- which(!(latitude > outliers_lat[1] & latitude < outliers_lat[2]))

# Replace outliers with means
total_training[xlat, c("latitude")]  <- c(mean(total_training$latitude))
total_training[xlon, c("longitude")] <- c(mean(total_training$longitude))

In [ ]:
# We´ll do the same for records where we have missing latitude and longitude:

# Replace missings with means
total_training[is.na(total_training$latitude), c("latitude")]   <- mean(total_training$latitude, na.rm = TRUE)
total_training[is.na(total_training$longitude), c("longitude")] <- mean(total_training$longitude, na.rm = TRUE)

In [ ]:
# Now, the last thing on geo-location attributes is clustering.

lat_long <- total_training[,c("longitude","latitude")]
cl <- kmeans(lat_long, length(unique(total_training$region_zip)))
total_training[,"kmeans_cluster"] <- cl$cluster

In [ ]:
names(total_training)
total_training

In [ ]:
# Now that we know about the location attributes, we can safely remove it and reduce the data.
total_training[,c("pos", "fips", "region_county", "region_city", "region_zip")] <- NULL

In [ ]:
summary(total_training)
paste("Dimensions of training_samples: " , toString(dim(total_training)))
# We can easily do na.execlude() and the data will reduce drastically, but that also means data loss. Which is not a desired thing. 
# Let us see that happens in that. 

summary(na.exclude(total_training))
paste("Dimensions of training_samples: " , toString(dim(na.exclude(total_training))))

In [ ]:
# Using the above summary, we can conclude that na.exclude() removes a lot of data, that might be wanted for our use. 
# First let us set null to 0 because the string columns are taken care of. Only numeric (double and integer) columns are left.
total_training[is.na(total_training)] <- 0

# Thus we will inspect each attribute individually.
# 1. lattitude - No change
# 2. longitude - No change
# 3. id_parcel - unique ID but with exception to 200 records that have duplicated id_parcel. There is no NA values
length(total_training$id_parcel)
length(unique(total_training$id_parcel))

# 4. logerror - log error (does not have any NA values)
# 5. date - converted the string to Date data type (earlier only)
# 6. num_bathroom and num_bathroom_calc - show same description and also same statistics 
#     The data varies from 1 to 18. Increases suddenly in the 4th quartile (outliers)
#     (We will remove the one with maximum NAs)
total_training$num_bathroom_calc <- NULL
# 7. Censustractandblock and rawcensustractandblock are ID <- remove
total_training$censustractandblock <- NULL
total_training$rawcensustractandblock <- NULL

In [ ]:
# 8. build_year and tax_year can be taken care of by subtracting 2019 from it to normalise it.
total_training$build_year <- 2019 - total_training$build_year
total_training$tax_year <- 2019 - total_training$tax_year

In [ ]:
# We have condensed the data considerably. We can now focus only on the data and its attributes
# that have strong correlation with each other. This will help in the linear regression analysis.
# 
correlationMatrix <- as.data.frame(corrplot(cor(total_training, use="complete.obs"), type="lower"))

# Thus we will now only keep the data with close relation to get good linear regression fit. 
x <- row.names(correlationMatrix)[abs(correlationMatrix$tax_property) > 0.50]
y <- row.names(correlationMatrix)[abs(correlationMatrix$num_bath) > 0.50]
z <- row.names(correlationMatrix)[abs(correlationMatrix$tax_building) > 0.50]
a <- row.names(correlationMatrix)[abs(correlationMatrix$tax_land) > 0.50]
b <- row.names(correlationMatrix)[abs(correlationMatrix$quality) > 0.50]
c <- row.names(correlationMatrix)[abs(correlationMatrix$num_room) > 0.50]
d <- row.names(correlationMatrix)[abs(correlationMatrix$longitude) > 0.50]

# Removing other columns to null them.
column_names_to_null <- base::setdiff(names(total_training), unique(union_all(a,b,c,d,x,y,z)))
dummy <- total_training
total_training[,c(column_names_to_null)] <- NULL

In [ ]:
# Normalising Tax_building
total_training$tax_building <- (total_training$tax_building - mean(total_training$tax_building)) / sd(total_training$tax_building)

In [ ]:
# Normalising Tax_land
total_training$tax_total <- (total_training$tax_total - mean(total_training$tax_total)) / sd(total_training$tax_total)

In [ ]:
# Normalising Tax_land
total_training$tax_land <- (total_training$tax_land - mean(total_training$tax_land)) / sd(total_training$tax_land)

In [ ]:
# Normalising Tax_property
total_training$tax_property <- (total_training$tax_property - mean(total_training$tax_property)) / sd(total_training$tax_property)

In [ ]:
# Creating model for making linear model - Tax Attributes
model1 <- lm(tax_building ~ tax_land + tax_property + tax_total, total_training)
print(model1)
summary(model1)

In [ ]:
# Creating model for making linear model - Room-Features Attributes
model2 <- lm(quality ~ area_total_calc + area_live_finished + num_bath + num_room + num_bathroom + num_bedroom, total_training)
print(model2)
summary(model2)